In [74]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
import numpy as np

In [4]:
# driver setup

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [39]:
def get_attr(node, attr):
    if node == None:
        return None
    if attr == 'text':
        return node.text
    return node.get(attr)

def scrape_page(source_text):
    soup = BeautifulSoup(source_text)
    data = []
    for row in soup.find('div', class_='member_directory').find_all('tr', class_='list member'):
        name = get_attr(row.find('div', class_='title').find('a'), 'text')
        chamber_profile_url = get_attr(row.find('div', class_='title').find('a'), 'href')

        info_table = row.find('table', class_='info')

        phone = get_attr(info_table.find('td'), 'text')
        email = get_attr(info_table.find('a', href=re.compile("mailto:")), 'href')
        website = get_attr(info_table.find('a', target='_blank'), 'href')
        fb = get_attr(info_table.find('a', class_='facebook'), 'href')
        twitter = get_attr(info_table.find('a', class_='twitter'), 'href')
        insta = get_attr(info_table.find('a', class_='instagram'), 'href')
        
        data.append([name, chamber_profile_url, phone, email, website, fb, twitter, insta])

    return data

In [44]:
def scrape_lincoln_park(drv):
    drv.get('https://www.lincolnparkchamber.com/directory/')
    button = drv.find_element_by_css_selector('button.list-display')
    button.click()
    time.sleep(3)
    more_pages = True
    data = []
    
    while more_pages:
        new_data = scrape_page(drv.page_source)
        data.append(new_data)
        try:
            next_button = drv.find_element_by_css_selector('span.right-arrow')
            next_button.click()
            time.sleep(3)
        except:
            more_pages = False

    return data

In [45]:
results = scrape_lincoln_park(driver)

In [68]:
results

[[['80 Proof',
   'https://www.lincolnparkchamber.com/member/80-proof/',
   '(773) 966-0404',
   'mailto:80proof@4cbars.com',
   'http://www.80proofchi.com',
   'https://www.facebook.com/80-Proof-Chi-1888405257847780/',
   'https://twitter.com/80proofchi',
   'https://www.instagram.com/80proofchi/'],
  ['AlphaGraphics',
   'https://www.lincolnparkchamber.com/member/alphagraphics/',
   '(312) 266-9266',
   'mailto:rhalverson@alphagraphics.com',
   'https://www.alphagraphics.com/centers/chicago-illinois-us715',
   'https://www.facebook.com/AlphaGraphicsChicagoNorth',
   'https://twitter.com/agchicagonorth',
   None],
  ['Ann & Robert H. Lurie Children’s Hospital of Chicago',
   'https://www.lincolnparkchamber.com/member/ann-robert-h-lurie-childrens-hospital-of-chicago/',
   '(312) 227-7089',
   None,
   'http://www.luriechildrens.org',
   'http://facebook.com/luriechildrens',
   'http://twitter.com/luriechildrens',
   None],
  ['Athletico Physical Therapy – Lincoln Park East',
   'https:

In [56]:
flattened = [row for page in results for row in page]

In [58]:
len(flattened)

485

In [63]:
lincoln_park = pd.DataFrame(flattened, columns=['name', 'chamber_profile_url', 'phone', 'email', 'website', 'fb', 'twitter', 'insta'])


In [71]:
lincoln_park.phone.value_counts()
# replace Email with nan
# cut 'mailto:' out of email
lincoln_park.email = lincoln_park.email.str.replace('mailto:', '')

In [76]:
lincoln_park.phone = lincoln_park.phone.replace(to_replace='Email', value=np.nan)

In [78]:
lincoln_park.to_csv('../data/processed/lincoln_park_chamber_scraped_0116.csv')

In [3]:
urls = [
    'https://www.lincolnparkchamber.com/directory/',
    'https://www.edgewater.org/membership-directory/',
    'https://www.lakeviewchamber.com/list'
]

In [ ]:
# use selenium
# vist site
# click list button
# loop through until next button disappears
# grab every class list member
# click next button